In [2]:
from sqlalchemy import create_engine
from urllib.parse import quote_plus
from sqlalchemy import text

# Replace these variables with your actual database details
server_name = 'LAPTOP-J2KGRRPR'
database_name = 'db'
username = 'Vera'
password = 'Vera@123'
encoded_password = quote_plus(password)
port=1433
# Create the connection string for the SQL Server
# connection_string = f'mssql+pymssql://{username}:{encoded_password}@{server_name}/{database_name}'
connection_string = f'mssql+pymssql://{username}:{encoded_password}@{server_name}:{port}'# Create the engine
engine = create_engine(connection_string)

# Connect to the server
connection = engine.connect()

# # Example query: Fetching the first 10 rows from a table named 'your_table'
# result = connection.execute(text("SELECT TOP 10 * FROM Orders"))
# for row in result:
#     print(row)

# # Close the connection
# connection.close()


In [10]:
from sqlalchemy import create_engine,inspect,text
from urllib.parse import quote_plus
from sqlalchemy import text

# Replace these variables with your actual database details
server_name = 'Nimish_'
database_name = 'mytest'
username = 'test'
password = 'test'
encoded_password = quote_plus(password)
port=1433
# Create the connection string for the SQL Server
# connection_string = f'mssql+pymssql://{username}:{encoded_password}@{server_name}/{database_name}'
connection_string = f'mssql+pymssql://{username}:{encoded_password}@{server_name}:{port}/{database_name}'# Create the engine
engine = create_engine(connection_string)

# Connect to the server
connection = engine.connect()

# Example query: Fetching the first 10 rows from a table named 'your_table'
# result = connection.execute(text("SELECT TOP 10 * FROM Orders"))
# rows = result.fetchall()  # fetches all rows
# row_count = len(rows)
# print(row_count)
# for row in result:
#     print(row)

# # Close the connection
# connection.close()


In [11]:
from sqlalchemy.exc import NoSuchTableError
from sqlalchemy import create_engine, inspect, text

# Create an inspector
inspector = inspect(engine)

# Function to get databases
def get_databases(engine):
    with engine.connect() as conn:
        result = conn.execute(text("SELECT name FROM sys.databases"))
        return [row[0] for row in result]

# Function to get schemas in a database
def get_schemas(inspector):
    return inspector.get_schema_names()

# Function to get tables and views in a schema
def get_tables_and_views(inspector, schema):
    tables = inspector.get_table_names(schema=schema)
    views = inspector.get_view_names(schema=schema)
    return tables, views

# Function to get columns of a table or view
def get_columns(inspector, schema, table_name):
    try:
        return inspector.get_columns(table_name, schema=schema)
    except NoSuchTableError:
        print(f"Skipping temporary table: {table_name}")
        return []

# Function to get primary keys of a table
def get_primary_keys(inspector, schema, table_name):
    try:
        pk_info = inspector.get_pk_constraint(table_name, schema=schema)
        return pk_info['constrained_columns']
    except NoSuchTableError:
        print(f"Skipping temporary table: {table_name}")
        return []

# Function to get foreign keys of a table
def get_foreign_keys(inspector, schema, table_name):
    try:
        fk_info = inspector.get_foreign_keys(table_name, schema=schema)
        return [{'column': fk['constrained_columns'], 'ref_table': fk['referred_table'], 'ref_column': fk['referred_columns']} for fk in fk_info]
    except NoSuchTableError:
        print(f"Skipping temporary table: {table_name}")
        return []

# Dictionary to store the database structure
database_structure = {}

# Get list of databases
databases = get_databases(engine)
db1 = input("Enter db name: ")
for db in databases:
    if db1 in db:
        database_structure[db] = {}
        print(f"Database: {db}")

        # Switch to the database
        connection_string_db = f'mssql+pymssql://{username}:{encoded_password}@{server_name}:{port}/{db}'
        engine_db = create_engine(connection_string_db)
        inspector_db = inspect(engine_db)
        
        # Get list of schemas
        schemas = get_schemas(inspector_db)
        for schema in schemas:
            database_structure[db][schema] = {'tables': {}, 'views': {}}
            print(f"  Schema: {schema}")

            # Get tables and views for each schema
            tables, views = get_tables_and_views(inspector_db, schema)
            print(f"    Tables in {schema}:")
            for table in tables:
                print(f"      {table}")
                # Get columns for each table
                columns = get_columns(inspector_db, schema, table)
                if columns:
                    database_structure[db][schema]['tables'][table] = {
                        'columns': {column['name']: str(column['type']) for column in columns},
                        'primary_keys': get_primary_keys(inspector_db, schema, table),
                        'foreign_keys': get_foreign_keys(inspector_db, schema, table)
                    }

            print(f"    Views in {schema}:")
            for view in views:
                print(f"      {view}")
                # Get columns for each view
                columns = get_columns(inspector_db, schema, view)
                if columns:
                    database_structure[db][schema]['views'][view] = {
                        'columns': {column['name']: str(column['type']) for column in columns},
                        'primary_keys': get_primary_keys(inspector_db, schema, view),
                        'foreign_keys': get_foreign_keys(inspector_db, schema, view)
                    }

        # Close the engine for the current database
        engine_db.dispose()

# Close the main engine
engine.dispose()

# Print the dictionary structure
import pprint
pprint.pprint(database_structure)


Database: mytest
  Schema: db_accessadmin
    Tables in db_accessadmin:
    Views in db_accessadmin:
  Schema: db_backupoperator
    Tables in db_backupoperator:
    Views in db_backupoperator:
  Schema: db_datareader
    Tables in db_datareader:
    Views in db_datareader:
  Schema: db_datawriter
    Tables in db_datawriter:
    Views in db_datawriter:
  Schema: db_ddladmin
    Tables in db_ddladmin:
    Views in db_ddladmin:
  Schema: db_denydatareader
    Tables in db_denydatareader:
    Views in db_denydatareader:
  Schema: db_denydatawriter
    Tables in db_denydatawriter:
    Views in db_denydatawriter:
  Schema: db_owner
    Tables in db_owner:
    Views in db_owner:
  Schema: db_securityadmin
    Tables in db_securityadmin:
    Views in db_securityadmin:
  Schema: dbo
    Tables in dbo:
      dept
      emp
    Views in dbo:
  Schema: guest
    Tables in guest:
    Views in guest:
  Schema: INFORMATION_SCHEMA
    Tables in INFORMATION_SCHEMA:
    Views in INFORMATION_SCHEMA:
  

In [7]:
from sqlalchemy.exc import NoSuchTableError
# Create an inspector
inspector = inspect(engine)

# Function to get databases
def get_databases(engine):
    with engine.connect() as conn:
        result = conn.execute(text("SELECT name FROM sys.databases"))
        return [row[0] for row in result]

# Function to get schemas in a database
def get_schemas(inspector):
    return inspector.get_schema_names()

# Function to get tables and views in a schema
def get_tables_and_views(inspector, schema):
    tables = inspector.get_table_names(schema=schema)
    views = inspector.get_view_names(schema=schema)
    return tables, views


# Function to get columns of a table or view
def get_columns(inspector, schema, table_name):
    try:
        return inspector.get_columns(table_name, schema=schema)
    except NoSuchTableError:
        print(f"Skipping temporary table: {table_name}")
        return []

# Dictionary to store the database structure
database_structure = {}

# Get list of databases
databases = get_databases(engine)
db1 = input("Enter db name: ")
for db in databases:
    if db1 in db:
        database_structure[db] = {}
        print(f"Database: {db}")

        # Switch to the database
        connection_string_db = f'mssql+pymssql://{username}:{encoded_password}@{server_name}:{port}/{db}'
        engine_db = create_engine(connection_string_db)
        inspector_db = inspect(engine_db)
        

        # Get list of schemas
        schemas = get_schemas(inspector_db)
        for schema in schemas:
            database_structure[db][schema] = {'tables': {}, 'views': {}}
            print(f"  Schema: {schema}")

            # Get tables and views for each schema
            tables, views = get_tables_and_views(inspector_db, schema)
            print(f"    Tables in {schema}:")
            for table in tables:
                print(f"      {table}")
                # Get columns for each table
                columns = get_columns(inspector_db,schema,table)
                if columns:
                    database_structure[db][schema]['tables'][table] = {column['name']: str(column['type']) for column in columns}

            print(f"    Views in {schema}:")
            for view in views:
                print(f"      {view}")
                # Get columns for each view
                columns = get_columns(inspector_db, schema, view)
                if columns:
                    database_structure[db][schema]['views'][view] = {column['name']: str(column['type']) for column in columns}

        # Close the engine for the current database
        engine_db.dispose()

# Close the main engine
engine.dispose()

# Print the dictionary structure
import pprint
pprint.pprint(database_structure)

Database: mytest
  Schema: db_accessadmin
    Tables in db_accessadmin:
    Views in db_accessadmin:
  Schema: db_backupoperator
    Tables in db_backupoperator:
    Views in db_backupoperator:
  Schema: db_datareader
    Tables in db_datareader:
    Views in db_datareader:
  Schema: db_datawriter
    Tables in db_datawriter:
    Views in db_datawriter:
  Schema: db_ddladmin
    Tables in db_ddladmin:
    Views in db_ddladmin:
  Schema: db_denydatareader
    Tables in db_denydatareader:
    Views in db_denydatareader:
  Schema: db_denydatawriter
    Tables in db_denydatawriter:
    Views in db_denydatawriter:
  Schema: db_owner
    Tables in db_owner:
    Views in db_owner:
  Schema: db_securityadmin
    Tables in db_securityadmin:
    Views in db_securityadmin:
  Schema: dbo
    Tables in dbo:
      dept
      emp
    Views in dbo:
  Schema: guest
    Tables in guest:
    Views in guest:
  Schema: INFORMATION_SCHEMA
    Tables in INFORMATION_SCHEMA:
    Views in INFORMATION_SCHEMA:
  

In [8]:
database_structure

{'mytest': {'db_accessadmin': {'tables': {}, 'views': {}},
  'db_backupoperator': {'tables': {}, 'views': {}},
  'db_datareader': {'tables': {}, 'views': {}},
  'db_datawriter': {'tables': {}, 'views': {}},
  'db_ddladmin': {'tables': {}, 'views': {}},
  'db_denydatareader': {'tables': {}, 'views': {}},
  'db_denydatawriter': {'tables': {}, 'views': {}},
  'db_owner': {'tables': {}, 'views': {}},
  'db_securityadmin': {'tables': {}, 'views': {}},
  'dbo': {'tables': {'dept': {'EmployeeID': 'SMALLINT',
     'DepartmentID': 'SMALLINT',
     'DepartmentName': 'NVARCHAR(50) COLLATE "SQL_Latin1_General_CP1_CI_AS"',
     'JobTitle': 'NVARCHAR(50) COLLATE "SQL_Latin1_General_CP1_CI_AS"',
     'HireDate': 'DATE',
     'Salary': 'INTEGER',
     'ManagerID': 'SMALLINT',
     'SupervisorID': 'SMALLINT'},
    'emp': {'EmployeeID': 'SMALLINT',
     'FirstName': 'NVARCHAR(50) COLLATE "SQL_Latin1_General_CP1_CI_AS"',
     'LastName': 'NVARCHAR(50) COLLATE "SQL_Latin1_General_CP1_CI_AS"',
     'DateOf

In [2]:
import re
def parse_table_description(input_dict):
    print('called parse_table_description',40*'-')
    input_string = input_dict[list(input_dict.keys())[0]]
    print(input_string)
    try:
        if '<end_of_description>' in input_string:
            # Define the regular expression pattern
            pattern = r"TABLE NAME: (\w+)\n(.*?)\n<end_of_description>"
        else:
            # Define the regular expression pattern
            pattern = r"TABLE NAME: (\w+)\n(.*?)\n"

        # Extract table name and content using regular expressions
        match = re.match(pattern, input_string, re.DOTALL)
        print(match)
        if match:
            table_name, content = match.groups()

            # Split content into lines and create a dictionary
            table_dict = {line.split(' : ')[0]: line.split(' : ')[1] for line in content.split('\n') if line}

            # Create the final dictionary
            output_dict = {table_name: table_dict}

            return output_dict
        else:
            return {list(input_dict.keys())[0]:{}}
    except:
        return {list(input_dict.keys())[0]:{}}

In [3]:
mydict={'emp': 'TABLE NAME: emp\nEmployeeID : Unique ID assigned to each employee, used to get unique employee\nFirstName : First Name of the employee\nLastName : Last Name of the employee\nDateOfBirth : Date of Birth of the employee\nGender : Gender of the employee\nNationality : Nationality of the employee\nAddress : Address of the employee\nCity : City of the employee\nState : State of the employee\nZipCode : Zip Code of the employee\nCountry : Country of the employee\nEmail : Email of the employee\nPhoneNumber : Phone Number of the employee'}

In [4]:
print(mydict)

{'emp': 'TABLE NAME: emp\nEmployeeID : Unique ID assigned to each employee, used to get unique employee\nFirstName : First Name of the employee\nLastName : Last Name of the employee\nDateOfBirth : Date of Birth of the employee\nGender : Gender of the employee\nNationality : Nationality of the employee\nAddress : Address of the employee\nCity : City of the employee\nState : State of the employee\nZipCode : Zip Code of the employee\nCountry : Country of the employee\nEmail : Email of the employee\nPhoneNumber : Phone Number of the employee'}


In [5]:
parse_table_description(mydict)

called parse_table_description ----------------------------------------
TABLE NAME: emp
EmployeeID : Unique ID assigned to each employee, used to get unique employee
FirstName : First Name of the employee
LastName : Last Name of the employee
DateOfBirth : Date of Birth of the employee
Gender : Gender of the employee
Nationality : Nationality of the employee
Address : Address of the employee
City : City of the employee
State : State of the employee
ZipCode : Zip Code of the employee
Country : Country of the employee
Email : Email of the employee
PhoneNumber : Phone Number of the employee
<re.Match object; span=(0, 94), match='TABLE NAME: emp\nEmployeeID : Unique ID assigned >


{'emp': {'EmployeeID': 'Unique ID assigned to each employee, used to get unique employee'}}

In [6]:
def parse_table_description(input_dict):
    print('called parse_table_description', 40 * '-')
    input_string = input_dict[list(input_dict.keys())[0]]
    print(input_string)

    try:
        # Define the regular expression pattern
        pattern = r"TABLE NAME: (\w+)\n(.*?)\n"

        # Extract table name and content using regular expressions
        match = re.search(pattern, input_string, re.DOTALL)

        if match:
            table_name, content = match.groups()

            # Split content into lines and create a dictionary
            table_dict = {line.split(' : ')[0].strip(): line.split(' : ')[1].strip() for line in content.split('\n') if line}

            # Create the final dictionary
            output_dict = {table_name: table_dict}

            return output_dict
        else:
            return {list(input_dict.keys())[0]: {}}
    except Exception as e:
        print(f"Error: {e}")
        return {list(input_dict.keys())[0]: {}}

In [7]:
parse_table_description(mydict)

called parse_table_description ----------------------------------------
TABLE NAME: emp
EmployeeID : Unique ID assigned to each employee, used to get unique employee
FirstName : First Name of the employee
LastName : Last Name of the employee
DateOfBirth : Date of Birth of the employee
Gender : Gender of the employee
Nationality : Nationality of the employee
Address : Address of the employee
City : City of the employee
State : State of the employee
ZipCode : Zip Code of the employee
Country : Country of the employee
Email : Email of the employee
PhoneNumber : Phone Number of the employee


{'emp': {'EmployeeID': 'Unique ID assigned to each employee, used to get unique employee'}}

In [9]:
from langchain_community.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(file_path=r"C:\Users\nisha\Downloads\Test_genpact.csv")

data = loader.load()

In [10]:
data

[Document(page_content='Question: What is Azure Key Vault Encryption?\nAnswer: Azure provides the transport layer security by ensuring any data fiow from azure key vault to client must be encrypted. When a user hits the key vault URI to get the key information, the response from the URI is encrypted while in transit. Besides this, when you store your key and secrets in a key vault all of them has been encrypted first and then stored. This is how it provides security at rest. Azure key vault stores the secrets and key with such a strong encryption that even Microsoft itself won’t be able to see your key and secrets in any way.', metadata={'source': 'C:\\Users\\nisha\\Downloads\\Test_genpact.csv', 'row': 0}),
 Document(page_content='Question: What is Microsoft Azure Sentinel?\nAnswer: : In organization cloud infrastructure there is always risk for threat, virus and other unauthorized or malicious activities. This can be spread out across the cloud infrastructures, on premises infrastruct

In [11]:
import re

def parse_table_description(input_dict):
    print('called parse_table_description', 40 * '-')
    input_string = input_dict[list(input_dict.keys())[0]]
    print(input_string)

    try:
        # Define the regular expression pattern to extract the table name and all subsequent content
        pattern = r"TABLE NAME: (\w+)\n(.*)"

        # Extract table name and content using regular expressions
        match = re.search(pattern, input_string, re.DOTALL)
        if match:
            table_name, content = match.groups()

            # Split content into lines and create a dictionary
            # Each line is expected to be in the format "ColumnName : Description"
            table_dict = {}
            for line in content.split('\n'):
                if line:  # Check if the line is not empty
                    parts = line.split(' : ')
                    if len(parts) == 2:  # Ensure that the line can be split into exactly two parts
                        key, value = parts
                        table_dict[key.strip()] = value.strip()

            # Create the final dictionary with the table name as the key and column descriptions as values
            output_dict = {table_name: table_dict}
            return output_dict
        else:
            return {list(input_dict.keys())[0]: {}}
    except Exception as e:
        print(f"Error: {e}")
        return {list(input_dict.keys())[0]: {}}

In [12]:
parse_table_description(mydict)

called parse_table_description ----------------------------------------
TABLE NAME: emp
EmployeeID : Unique ID assigned to each employee, used to get unique employee
FirstName : First Name of the employee
LastName : Last Name of the employee
DateOfBirth : Date of Birth of the employee
Gender : Gender of the employee
Nationality : Nationality of the employee
Address : Address of the employee
City : City of the employee
State : State of the employee
ZipCode : Zip Code of the employee
Country : Country of the employee
Email : Email of the employee
PhoneNumber : Phone Number of the employee


{'emp': {'EmployeeID': 'Unique ID assigned to each employee, used to get unique employee',
  'FirstName': 'First Name of the employee',
  'LastName': 'Last Name of the employee',
  'DateOfBirth': 'Date of Birth of the employee',
  'Gender': 'Gender of the employee',
  'Nationality': 'Nationality of the employee',
  'Address': 'Address of the employee',
  'City': 'City of the employee',
  'State': 'State of the employee',
  'ZipCode': 'Zip Code of the employee',
  'Country': 'Country of the employee',
  'Email': 'Email of the employee',
  'PhoneNumber': 'Phone Number of the employee'}}

In [13]:
str1 = "Hello World"
is_present = "World" in str1
is_present

True

In [15]:
a = 5
b = 6
a = a + b 
print(a)
b = a - b
print("b",b)
a = a - b
print("a",a)

11
b 5
a 6
